In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.3-bin-hadoop2.7/')
import pyspark
from pyspark.sql import SparkSession

In [2]:
## Spark DataFrames GroupBy and Aggregate
## groupBy allows you to group rows together based off some column value
## For example you can group together sales data by the day the sale occurred 
## Once you performed groupBy operation you can then use some sort of aggregate method or function off of that data
## All an aggregate function does is that it combines or aggregates multiple rows of data into a single output. 
## An example would be taking the sum of a bunch of numerical inputs or counting the number of inputs. 
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [3]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [5]:
df.printSchema() # only one numerical column

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [7]:
df.groupBy("Company")

In [8]:
df.groupBy('Company').mean()

DataFrame[Company: string, avg(Sales): double]

In [9]:
df.groupBy('Company').mean().show() ## sum, max, min, count etc..

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [10]:
df.agg({'Sales':'sum'}).show() ## We pass in the column that we want to aggregate and then the actual function
## we want to use as a string.

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [11]:
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [12]:
group_data = df.groupBy('Company')

In [13]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [15]:
# how to import functions in spark
from pyspark.sql.functions import countDistinct, avg,stddev

In [18]:
df.select(countDistinct('Sales')).show() ## it counts the distinct number of sales values

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [19]:
df.select(avg('Sales')).show()  ## avg sales

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [21]:
df.select(avg('Sales').alias('Average Sales')).show()  ## we can give an alias

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [22]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [23]:
from pyspark.sql.functions import format_number # we dont like the column name and the digits are many

In [26]:
sales_std = df.select(stddev("Sales").alias('std'))

In [28]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [32]:
sales_std.select(format_number('std', 2).alias('Std')).show() # we formatted the digit so that it only gets two digits


+------+
|   Std|
+------+
|250.09|
+------+



In [33]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [35]:
df.orderBy('Sales').show() # ascending order

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [36]:
df.orderBy(df['Sales'].desc()).show() # descending order.

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

